[1581. Customer Who Visited but Did Not Make Any Transactions](https://leetcode.com/problems/customer-who-visited-but-did-not-make-any-transactions/?envType=study-plan-v2&envId=top-sql-50)

In [2]:
import findspark
findspark.init()

import datetime

In [3]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

spark = SparkSession.builder.master("local").appName("playground").getOrCreate()

In [4]:
visits_schema = T.StructType(
    [
        T.StructField("visit_id", T.IntegerType(), False),
        T.StructField("customer_id", T.IntegerType(), False)
    ]
)

transactions_schema = T.StructType(
    [
        T.StructField("transaction_id", T.IntegerType(), False),
        T.StructField("visit_id", T.IntegerType(), False),
        T.StructField("amount", T.IntegerType(), False)
    ]
)

visits_data = [
    (1, 23),
    (2, 9),
    (4, 30),
    (5, 54),
    (6, 96),
    (7, 54),
    (8, 54)
]

transactions_data = [
    (2, 5, 310),
    (3, 5, 300),
    (9, 5, 200),
    (12, 1, 910),
    (13, 2, 970)
]

visits = spark.createDataFrame(visits_data, visits_schema)
visits.show()

transactions = spark.createDataFrame(transactions_data, transactions_schema)
transactions.show()

+--------+-----------+
|visit_id|customer_id|
+--------+-----------+
|       1|         23|
|       2|          9|
|       4|         30|
|       5|         54|
|       6|         96|
|       7|         54|
|       8|         54|
+--------+-----------+

+--------------+--------+------+
|transaction_id|visit_id|amount|
+--------------+--------+------+
|             2|       5|   310|
|             3|       5|   300|
|             9|       5|   200|
|            12|       1|   910|
|            13|       2|   970|
+--------------+--------+------+



In [8]:
visit_txn = (visits
             .join(transactions, ["visit_id"], "left")
             .select("customer_id", "transaction_id"))

visit_txn.show()

+-----------+--------------+
|customer_id|transaction_id|
+-----------+--------------+
|         23|            12|
|         96|          null|
|         54|             2|
|         54|             3|
|         54|             9|
|         30|          null|
|         54|          null|
|         54|          null|
|          9|            13|
+-----------+--------------+



In [11]:
null_visit_txn = visit_txn.where("transaction_id is null")
null_visit_txn.show()


+-----------+--------------+
|customer_id|transaction_id|
+-----------+--------------+
|         96|          null|
|         30|          null|
|         54|          null|
|         54|          null|
+-----------+--------------+



In [13]:
(null_visit_txn
 .groupBy("customer_id")
 .agg(F.count("customer_id")
      .alias("count_no_trans"))
 .show())

+-----------+--------------+
|customer_id|count_no_trans|
+-----------+--------------+
|         54|             2|
|         96|             1|
|         30|             1|
+-----------+--------------+

